In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StringType, BooleanType, LongType, DoubleType
import json

In [ ]:
spark = SparkSession.builder \
    .appName("Lendo arquivos CSV") \
    .getOrCreate()

In [ ]:
def _build_schema(schema_arg: str):
    d_types = {
        "string": StringType(),
        "long": LongType(),
        "double": DoubleType(),
        "boll": BooleanType()
    }
    
    split_values = schema_arg.split(",")
    schema = StructType()
    
    for word in split_values:
        x = word.split(":")
        schema.add(x[0], d_types[x[1]], True)
    
    return schema

In [ ]:
def _get_schema_data(name: str) -> str:
    with open(f"./schemas/{name}.json") as arquivo:
        schema = json.load(arquivo)
    return ",".join(map(lambda p: f"{p['name']}:{p['type']}", schema['fields']))

In [ ]:
def _join_files(table: str):
    df = spark.read.options(sep=";",encoding="latin1") \
                .schema(_build_schema(_get_schema_data(table))) \
                .csv(f"./data/bronze/{table}")
    df.repartition(1).write.mode("overwrite").parquet(f"./data/silver/{table}")

In [ ]:
_join_files("Empresas")